In [ ]:
import pandas as pd

In [ ]:
pwd

'/content'

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd 
import io
df2 = pd.read_csv(io.BytesIO(uploaded['tweet_content_010921.csv']))

In [ ]:
df2

,Unnamed: 0,id,text,start,end
0,0,1425623064121724929,RT @WClementeIII: 76.8% of Bitcoin's money sup...,2018-01-01 00:00:00+00:00,2018-01-02T00:00:00Z
1,1,1425622402176675842,RT @1MillionTokens: 🥰1000 #1MT #Giveaway ( wor...,2018-01-01 00:00:00+00:00,2018-01-02T00:00:00Z
2,2,1425622403439267849,"RT @gladstein: 1/ My essay ""Inside Cuba's Bitc...",2018-01-01 00:00:00+00:00,2018-01-02T00:00:00Z
3,3,1425622403695058947,RT @AirdropDet: 🔍 New #Airdrop: META 1 Coin\n\...,2018-01-01 00:00:00+00:00,2018-01-02T00:00:00Z
4,4,1425622404194136066,RT @RewardHunters: Want to know about The Batt...,2018-01-01 00:00:00+00:00,2018-01-02T00:00:00Z
...,...,...,...,...,...
413419,413419,1425622715155853315,Finally got ahold of the ₿ symbol. \n\n#Bitcoin,2020-07-25 00:00:00+00:00,2020-07-26T00:00:00Z
413420,413420,1425622716145672193,RT @Hansan62107581: You Must be Heard About Su...,2020-07-25 00:00:00+00:00,2020-07-26T00:00:00Z
413421,413421,1425622719782137860,#CHINA #amcstock #AMCSQUEEZE #ADAM $GLD $SLV $...,2020-07-25 00:00:00+00:00,2020-07-26T00:00:00Z
413422,413422,1425622698353364994,RT @miguelmazzinni: Miguel Rivas:\n¿Qué les di...,2020-07-25 00:00:00+00:00,2020-07-26T00:00:00Z


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 46.3 MB/s 
     |████████████████████████████████| 636 kB 30.3 MB/s 
     |████████████████████████████████| 895 kB 45.3 MB/s 
     |████████████████████████████████| 50 kB 4.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []


    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)



OSError: ignored

In [27]:

def sentimentscore(text):
  # text = "Good night 😊"
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  # # TF
  # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
  # model.save_pretrained(MODEL)

  # text = "Good night 😊"
  # encoded_input = tokenizer(text, return_tensors='tf')
  # output = model(encoded_input)
  # scores = output[0][0].numpy()
  # scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
      l = labels[ranking[i]]
      s = scores[ranking[i]]
      # print(f"{i+1}) {l} {np.round(float(s), 4)}")
      
  return (scores[2]-scores[0])/scores[1]


In [ ]:
for i in range(413424):
  df2['sentimentscore'][i]=sentimentscore(df2.text[i])

df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
